# **Spark Apache (Семинары)**
## **Урок 4. Изучение Spark Structure Streaming**
### Задание:
1. Условие: Используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел

In [ ]:
! pip install pyspark >> /dev/null

In [ ]:
# Установка требуемых модулей
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, lit, row_number, rank
#from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [ ]:
# Установка сессии spark
spark = SparkSession.builder.appName('RateOdd').getOrCreate()

# Объявляем поток чтения из RATE источника
df_input = spark.readStream.format("rate").option("rowsPerSecond", 3).load()

# Объявляем поток записи
odd_sum_query = df_input \
    .filter(col('value') % 2 != 0) \
    .groupBy(F.window("timestamp", "1 seconds")) \
        .sum('value') \
        .withColumnRenamed('sum(value)', 'sum_odd') \
    .writeStream \
    .outputMode("update") \
    .queryName("aggregates") \
    .format("memory") \

# Запускаем поток
odd_sum_query.start().awaitTermination(15)

spark.sql("select * from aggregates").show(50, truncate=False)

spark.stop()

+------------------------------------------+-------+
|window                                    |sum_odd|
+------------------------------------------+-------+
|{2024-04-26 10:12:29, 2024-04-26 10:12:30}|15     |
|{2024-04-26 10:12:30, 2024-04-26 10:12:31}|36     |
|{2024-04-26 10:12:27, 2024-04-26 10:12:28}|9      |
|{2024-04-26 10:12:24, 2024-04-26 10:12:25}|1      |
|{2024-04-26 10:12:26, 2024-04-26 10:12:27}|12     |
|{2024-04-26 10:12:25, 2024-04-26 10:12:26}|3      |
|{2024-04-26 10:12:28, 2024-04-26 10:12:29}|24     |
+------------------------------------------+-------+

